# prerequisites :
 - request a visu node on which the notebook will be launched : 
```
salloc --ntasks=1 --cpus-per-task=5 --account=egi@cpu --partition=prepost --time=03:10:00 srun --pty bash
```
or go to https://jupyterhub.idris.fr and select python3.8.8 kernel

 - all the input files (coordinates, domain, bathymetry, initial conditions, damping files, runoffs, etc ..) needed to run NEMO are stored in : ```/gpfswork/rech/eee/rote001/nemo4-imhotep-main/cfgs/MED025.L75-JZAA001/INPUTS```
 - all the forcing files (atmospheric forcings +weights) needed to run NEMO are store in : ```/gpfswork/rech/eee/rote001/nemo4-imhotep-main/cfgs/MED025.L75-JZAA001/FORCING```
 - you have compiled NEMO code and the executable is here : ```/gpfswork/rech/eee/rote001/nemo4-imhotep-main/cfgs/MED025.L75-JZAA001/BLD/bin/nemo.exe```
 - you have compiled XIOS code and the excutable is here : ```/gpfswork/rech/cli/rcst991/XIOS/bin/xios_server.exe```
 - the namelists are available at : ```/gpfswork/rech/eee/rote001/nemo4-imhotep-main/cfgs/MED025.L75-JZAA001/EXP_SMSIM/```
 - the xml files are available at : ```/gpfswork/rech/eee/rote001/nemo4-imhotep-main/cfgs/MED025.L75-JZAA001/EXPREF/```

In [13]:
from glob import glob
from smartsim import Experiment
from smartsim.database import Orchestrator
from smartsim.log import log_to_file
import warnings

In [14]:
def create_distributed_orchestrator(
    exp,
    orchestrator_port,
    orchestrator_interface,
    orchestrator_nodes,
    orchestrator_nemo_node_features,
    walltime
    ):

    orchestrator = exp.create_database(
        port = orchestrator_port,
        interface = orchestrator_interface,
        db_nodes = orchestrator_nodes,
        time=walltime,
        threads_per_queue=2,
        batch=True
    )
    
    orchestrator.set_cpus(18)
    orchestrator.set_batch_arg("account", "egi@cpu")
    orchestrator.set_batch_arg("exclusive",None)
    orchestrator.set_batch_arg("partition","compil")
    
    return orchestrator

In [15]:
def create_nemo_model(
        experiment,
        nemo_num_nodes,
        nemo_tasks_per_node,
        nemo_node_features,
        xios_num_nodes,
        xios_tasks_per_node,
        xios_node_features,
        xios_exe,
        nemo_cfg_path,
        nemo_input_path,
        nemo_forcing_path,
        walltime
    ):

    # Specify the job settings for jean-zay
    batch_args = {
        "ntasks":40,
        "partition": "compil",
        "export":"ALL"
    }
    nemo_batch_settings = experiment.create_batch_settings(
        nodes=nemo_num_nodes+xios_num_nodes,
        time=walltime,
        account="egi@cpu",
        batch_args=batch_args
    )
    nemo_batch_settings.add_preamble('source /linkhome/rech/genige01/rote001/smart_path.ksh; module load intel-all/2019.4; module load hdf5/1.10.5-mpi; module load netcdf/4.7.2-mpi; ldd /gpfswork/rech/eee/rote001/nemo4-imhotep-main/cfgs/MED025.L75-JZAA001/BLD/bin/nemo.exe')
    # Create NEMO portion of the MPMD job
    nemo_run_settings = experiment.create_run_settings(
        f"{nemo_cfg_path}/BLD/bin/nemo.exe",
        run_command="srun"
    )

    nemo_run_settings.set_tasks_per_node(nemo_tasks_per_node)
    nemo_run_settings.set_tasks(nemo_num_nodes*nemo_tasks_per_node)
    # Create a 1-member ensemble to launch the model

    nemo_model = experiment.create_ensemble(
        "nemo",
        batch_settings = nemo_batch_settings,
        run_settings   = nemo_run_settings,
        replicas=1
    )
    xml_files = glob(f"{nemo_cfg_path}/EXPREF/*.xml")
    namelist_files = glob(f"{nemo_cfg_path}/EXP_SMSIM/namelist*")
    forcing_files = glob(f"{nemo_forcing_path}/*")
    forcing_files+= glob(f"{nemo_input_path}/*")

    nemo_model.attach_generator_files(
        to_configure=namelist_files,
        to_copy=xml_files,
        to_symlink=forcing_files
    )

    return nemo_model

In [16]:
def configure_nemo_model(
    ensemble,
    job_number,
    first_time_step,
    last_time_step,
    restart_flag,
    restart_directory
    ):

    nemo_config_options = {
        "NN_NO": job_number,
        "NIT000": first_time_step,
        "NITEND": last_time_step,
        "RESTART": restart_flag,
        "CN_DIRRST": restart_directory
    }

    for model in ensemble: 
        model.params = nemo_config_options
        model.register_incoming_entity(model)

    return ensemble

In [17]:
#params
nemo_num_nodes=1
nemo_tasks_per_node=40
nemo_node_features=False
xios_num_nodes=0
xios_tasks_per_node=0
xios_node_features=''
nemo_cfg_path="/gpfswork/rech/eee/rote001/nemo4-imhotep-main/cfgs/MED025.L75-JZAA001"
nemo_forcing_path="/gpfswork/rech/eee/rote001/nemo4-imhotep-main/cfgs/MED025.L75-JZAA001/FORCING"
nemo_input_path="/gpfswork/rech/eee/rote001/nemo4-imhotep-main/cfgs/MED025.L75-JZAA001/MY_INPUTS"
xios_exe="/gpfswork/rech/cli/rcst991/XIOS/bin/xios_server.exe"
job_number = 1
first_time_step = 1
last_time_step = 6000
restart_flag = ".false."
restart_directory = "/gpfswork/rech/eee/rote001/nemo4-imhotep-main/cfgs/MED025.L75-JZAA001"
orchestrator_port=6780
orchestrator_interface='ib0'
orchestrator_nodes=1
orchestrator_nemo_node_features=False
walltime="00:15:00"

In [18]:
experiment = Experiment("NEMO-MEDITERR", launcher="slurm")


In [19]:
    nemo_model = create_nemo_model(
        experiment,
        nemo_num_nodes,
        nemo_tasks_per_node,
        nemo_node_features,
        xios_num_nodes,
        xios_tasks_per_node,
        xios_node_features,
        xios_exe,
        nemo_cfg_path,
        nemo_input_path,
        nemo_forcing_path,
        walltime
     )


In [20]:
    configure_nemo_model(
        nemo_model,
        job_number,
        first_time_step,
        last_time_step,
        restart_flag,
        restart_directory
    )


In [21]:
    orchestrator = create_distributed_orchestrator(
        experiment,
        orchestrator_port,
        orchestrator_interface,
        orchestrator_nodes,
        orchestrator_nemo_node_features,
        walltime
    )



In [22]:
    experiment.generate( nemo_model, orchestrator, overwrite=True )


14:28:56 jean-zay-pp1 SmartSim[2732542] INFO Working in previously created experiment
14:28:57 jean-zay-pp1 SmartSim[2732542] WARNING Unused tag  partial slip (0-2)  on line(s): [57]
14:28:57 jean-zay-pp1 SmartSim[2732542] WARNING Unused tag  partial slip (0-2)  on line(s): [57]


In [23]:
    experiment.start(nemo_model, orchestrator, summary=True)


14:28:57 jean-zay-pp1 SmartSim[2732542] INFO 

=== Launch Summary ===
Experiment: NEMO-MEDITERR
Experiment Path: /gpfsdswork/projects/rech/eee/rote001/nemo4-imhotep-main/cfgs/MED025.L75-JZAA001/EXP_SSINT/NEMO-MEDITERR
Launcher: slurm
Ensembles: 1
Database Status: launching

=== Ensembles ===
nemo
Members: 1
Batch Launch: True
Batch Command: sbatch
Batch arguments:
	ntasks = 40
	partition = compil
	export = ALL
	nodes = 1
	time = 00:15:00
	account = egi@cpu

=== Database ===
Shards: 1
Port: 6780
Network: ib0
Batch Launch: True
Batch Command: sbatch
Batch arguments:
	nodes = 1
	time = 00:15:00
	cpus-per-task = 18
	account = egi@cpu
	exclusive = None
	partition = compil




14:29:14 jean-zay-pp1 SmartSim[2732542] INFO Orchestrator launched as a batch
14:29:14 jean-zay-pp1 SmartSim[2732542] INFO While queued, SmartSim will wait for Orchestrator to run
14:29:14 jean-zay-pp1 SmartSim[2732542] INFO CTRL+C interrupt to abort and cancel launch
14:29:43 jean-zay-pp1 SmartSim[2732542] INFO nemo(1696550): New
14:29:48 jean-zay-pp1 SmartSim[2732542] INFO nemo(1696550): Paused
14:29:53 jean-zay-pp1 SmartSim[2732542] INFO nemo(1696550): Paused
14:29:58 jean-zay-pp1 SmartSim[2732542] INFO nemo(1696550): Paused
14:30:03 jean-zay-pp1 SmartSim[2732542] INFO nemo(1696550): Paused
14:30:08 jean-zay-pp1 SmartSim[2732542] INFO nemo(1696550): Paused
14:30:13 jean-zay-pp1 SmartSim[2732542] INFO nemo(1696550): Paused
14:30:18 jean-zay-pp1 SmartSim[2732542] INFO nemo(1696550): Running
14:30:23 jean-zay-pp1 SmartSim[2732542] INFO nemo(1696550): Running
14:30:28 jean-zay-pp1 SmartSim[2732542] INFO nemo(1696550): Completed
14:30:28 jean-zay-pp1 SmartSim[2732542] INFO nemo(1696550):

In [24]:
    experiment.stop(orchestrator)

14:30:33 jean-zay-pp1 SmartSim[2732542] INFO Stopping model orchestrator with job name orchestrator-CP7J5COV7FNU


In [25]:
!ls NEMO-MEDITERR/nemo

nemo_0	nemo.err  nemo.out  nemo.sh


In [26]:
!cat NEMO-MEDITERR/nemo/nemo.sh

#!/bin/bash

#SBATCH --output=/gpfsdswork/projects/rech/eee/rote001/nemo4-imhotep-main/cfgs/MED025.L75-JZAA001/EXP_SSINT/NEMO-MEDITERR/nemo/nemo.out
#SBATCH --error=/gpfsdswork/projects/rech/eee/rote001/nemo4-imhotep-main/cfgs/MED025.L75-JZAA001/EXP_SSINT/NEMO-MEDITERR/nemo/nemo.err
#SBATCH --job-name=nemo-CP7J5P8RLKWX
#SBATCH --ntasks=40
#SBATCH --partition=compil
#SBATCH --export=ALL
#SBATCH --nodes=1
#SBATCH --time=00:15:00
#SBATCH --account=egi@cpu
source /linkhome/rech/genige01/rote001/smart_path.ksh; module load intel-all/2019.4; module load hdf5/1.10.5-mpi; module load netcdf/4.7.2-mpi; ldd /gpfswork/rech/eee/rote001/nemo4-imhotep-main/cfgs/MED025.L75-JZAA001/BLD/bin/nemo.exe

cd /gpfsdswork/projects/rech/eee/rote001/nemo4-imhotep-main/cfgs/MED025.L75-JZAA001/EXP_SSINT/NEMO-MEDITERR/nemo/nemo_0 ; /gpfslocalsys/slurm/current/bin/srun --output /gpfsdswork/projects/rech/eee/rote001/nemo4-imhotep-main/cfgs/MED025.L75-JZAA001/EXP_SSINT/NEMO-MEDITERR/nemo/nemo_0/nemo_0.out --error /gp

In [27]:
!cat NEMO-MEDITERR/nemo/nemo.out

	linux-vdso.so.1 (0x000014eaceec5000)
	libstdc++.so.6 => /lib64/libstdc++.so.6 (0x000014eace906000)
	libsmartredis.so => /gpfswork/rech/eee/rote001/git/smartredis/install/lib/libsmartredis.so (0x000014eace60d000)
	libnetcdff.so.7 => /gpfslocalsup/spack_soft/netcdf-fortran/4.5.2/intel-19.0.4-knfz4zgnj6ohnfqulsg2z55orygeb7we/lib/libnetcdff.so.7 (0x000014eace044000)
	libnetcdf.so.15 => /gpfslocalsup/spack_soft/netcdf/4.7.2/intel-19.0.4-cqo7jj3yxaprhm23gr2tfq2f4epw7k3r/lib/libnetcdf.so.15 (0x000014eacdd0a000)
	libhdf5_hl.so.100 => /gpfslocalsup/spack_soft/hdf5/1.10.5/intel-19.0.4-lnysdcbocfokaq4yxc72wiplpfknw7e6/lib/libhdf5_hl.so.100 (0x000014eacdae0000)
	libhdf5.so.103 => /gpfslocalsup/spack_soft/hdf5/1.10.5/intel-19.0.4-lnysdcbocfokaq4yxc72wiplpfknw7e6/lib/libhdf5.so.103 (0x000014eacd406000)
	libz.so.1 => /lib64/libz.so.1 (0x000014eacd1ee000)
	libmpifort.so.12 => /gpfslocalsys/intel/parallel_studio_xe_2019_update4_cluster_edition/compilers_and_libraries_2019.4.243/linux/mpi/intel64/lib/l

In [28]:
!cat NEMO-MEDITERR/nemo/nemo.err

Loading intel-all/2019.4
  Loading requirement: intel-compilers/19.0.4 intel-mkl/2019.4 intel-mpi/2019.4
    intel-vtune/2019.4 intel-advisor/2019.4 intel-tbb/2019.6 intel-itac/2019.4
srun: error: jean-zay-pp4: tasks 0-39: Exited with exit code 127
srun: launch/slurm: _step_signal: Terminating StepId=1696550.0


In [29]:
!cat NEMO-MEDITERR/nemo/nemo_0/nemo_0.out

In [30]:
!cat NEMO-MEDITERR/nemo/nemo_0/nemo_0.err

/gpfswork/rech/eee/rote001/nemo4-imhotep-main/cfgs/MED025.L75-JZAA001/BLD/bin/nemo.exe: error while loading shared libraries: libnetcdff.so.7: cannot open shared object file: No such file or directory
/gpfswork/rech/eee/rote001/nemo4-imhotep-main/cfgs/MED025.L75-JZAA001/BLD/bin/nemo.exe: error while loading shared libraries: libnetcdff.so.7: cannot open shared object file: No such file or directory
/gpfswork/rech/eee/rote001/nemo4-imhotep-main/cfgs/MED025.L75-JZAA001/BLD/bin/nemo.exe: error while loading shared libraries: libnetcdff.so.7: cannot open shared object file: No such file or directory
/gpfswork/rech/eee/rote001/nemo4-imhotep-main/cfgs/MED025.L75-JZAA001/BLD/bin/nemo.exe: error while loading shared libraries: libnetcdff.so.7: cannot open shared object file: No such file or directory
/gpfswork/rech/eee/rote001/nemo4-imhotep-main/cfgs/MED025.L75-JZAA001/BLD/bin/nemo.exe: error while loading shared libraries: libnetcdff.so.7: cannot open shared object file: No such file or direc

In [31]:
!ls /gpfslocalsup/spack_soft/netcdf-fortran/4.5.2/intel-19.0.4-knfz4zgnj6ohnfqulsg2z55orygeb7we/lib/libnetcdff.so.7

/gpfslocalsup/spack_soft/netcdf-fortran/4.5.2/intel-19.0.4-knfz4zgnj6ohnfqulsg2z55orygeb7we/lib/libnetcdff.so.7
